# Invasive Species

In [1]:
path = 'data/'

## Create training and validation sets

In [ ]:
%cd data/train
%mkdir 0
%mkdir 1

In [2]:
import csv
import math
import os
import numpy as np
from glob import glob

In [ ]:
g = glob('*.jpg')
with open('../train_labels.csv') as csvfile:
    labels = csv.reader(csvfile, delimiter='\n')
    for row in labels:
        columns = row[0].split(',')
        if columns[0] != 'name':
            if columns[1] == '0':
                os.rename(columns[0] + '.jpg', '0/{}.jpg'.format(columns[0]))
            else:
                os.rename(columns[0] + '.jpg', '1/{}.jpg'.format(columns[0]))

In [ ]:
%mkdir ../valid
%mkdir ../valid/0
%mkdir ../valid/1
%cd 0

In [ ]:
g = glob('*.jpg')
np.random.shuffle(g)
for i in range(math.floor(len(g) / 10)):
    os.rename(g[i], '../../valid/0/' + g[i])

In [ ]:
%cd ../1

In [ ]:
g = glob('*.jpg')
np.random.shuffle(g)
for i in range(math.floor(len(g) / 10)):
    os.rename(g[i], '../../valid/1/' + g[i])

In [ ]:
%cd ../../..

## Set up VGG model

In [3]:
batch_size = 32

In [4]:
from vgg16 import Vgg16

Using Theano backend.
/usr/local/lib/python3.5/dist-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Preallocating 3633/4037 Mb (0.900000) on cuda1
Mapped name None to device cuda1: GeForce GTX 970 (0000:02:00.0)


In [5]:
vgg = Vgg16()

In [ ]:
batches = vgg.get_batches(path + 'train', batch_size=batch_size)
valid_batches = vgg.get_batches(path + 'valid', batch_size=batch_size)

In [6]:
vgg.finetune()

In [7]:
vgg.model.load_weights(path + 'finetune1.h5')

In [ ]:
vgg.fit(batches, math.floor(2067/batch_size), epochs=20, 
        validation_batches=valid_batches, 
        validation_steps=math.floor(228/batch_size))

In [ ]:
vgg.model.save_weights(path + 'finetune1.h5')

### Finetune earlier layers

In [ ]:
for layer in vgg.model.layers[12:]:
    layer.trainable = True

In [ ]:
vgg.fit(batches, math.floor(2067/batch_size), epochs=20, 
        validation_batches=valid_batches, 
        validation_steps=math.floor(228/batch_size))

In [ ]:
for layer in vgg.model.layers[10:]:
    layer.trainable = True

In [ ]:
vgg.fit(batches, math.floor(2067/batch_size), epochs=5, 
        validation_batches=valid_batches, 
        validation_steps=math.floor(228/batch_size))

In [ ]:
for layer in vgg.model.layers[8:]:
    layer.trainable = True

In [ ]:
vgg.fit(batches, math.floor(2067/batch_size), epochs=10, 
        validation_batches=valid_batches, 
        validation_steps=math.floor(228/batch_size))

In [ ]:
for layer in vgg.model.layers[4:]:
    layer.trainable = True

In [ ]:
vgg.fit(batches, math.floor(2067/batch_size), epochs=10, 
        validation_batches=valid_batches, 
        validation_steps=math.floor(228/batch_size))

In [ ]:
for layer in vgg.model.layers:
    layer.trainable = True

In [ ]:
vgg.fit(batches, math.floor(2067/batch_size), epochs=10, 
        validation_batches=valid_batches, 
        validation_steps=math.floor(228/batch_size))

In [ ]:
vgg.model.save_weights(path + 'finetune1.h5')

## Fix overfitting

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
gen = ImageDataGenerator(horizontal_flip=True, zoom_range=0.2, shear_range=0.2, 
                         width_shift_range=0.2, 
                         height_shift_range=0.2,)
batches = vgg.get_batches(path + 'train', batch_size=batch_size, gen=gen)
valid_batches = vgg.get_batches(path + 'valid', batch_size=batch_size, gen=gen)

Found 2067 images belonging to 2 classes.
Found 228 images belonging to 2 classes.


In [ ]:
vgg.fit(batches, math.floor(2067/batch_size), epochs=30,
        validation_batches=valid_batches, 
        validation_steps=math.floor(228/batch_size))

Epoch 1/30
64/64 [==============================] - 39s - loss: 0.2432 - acc: 0.8955 - val_loss: 0.2820 - val_acc: 0.8705
Epoch 2/30
64/64 [==============================] - 34s - loss: 0.2311 - acc: 0.9127 - val_loss: 0.3013 - val_acc: 0.8827
Epoch 3/30
64/64 [==============================] - 34s - loss: 0.2329 - acc: 0.8922 - val_loss: 0.2336 - val_acc: 0.9031
Epoch 4/30
64/64 [==============================] - 34s - loss: 0.2420 - acc: 0.8966 - val_loss: 0.2159 - val_acc: 0.9082
Epoch 5/30
64/64 [==============================] - 34s - loss: 0.2559 - acc: 0.8903 - val_loss: 0.2293 - val_acc: 0.9133
Epoch 6/30
64/64 [==============================] - 34s - loss: 0.2522 - acc: 0.8961 - val_loss: 0.2739 - val_acc: 0.8776
Epoch 7/30
64/64 [==============================] - 34s - loss: 0.2418 - acc: 0.8950 - val_loss: 0.2218 - val_acc: 0.9031
Epoch 8/30
64/64 [==============================] - 34s - loss: 0.2347 - acc: 0.8976 - val_loss: 0.2257 - val_acc: 0.9133
Epoch 9/30
64/64 [======